---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

In [17]:
print(pd.show_versions())


INSTALLED VERSIONS
------------------
commit: None
python: 3.6.2.final.0
python-bits: 64
OS: Linux
OS-release: 4.14.70-67.55.amzn1.x86_64
machine: x86_64
processor: x86_64
byteorder: little
LC_ALL: en_US.UTF-8
LANG: en_US.UTF-8
LOCALE: en_US.UTF-8

pandas: 0.19.2
nose: None
pip: 9.0.1
setuptools: 36.2.2.post20170724
Cython: 0.25.2
numpy: 1.12.1
scipy: 0.19.1
statsmodels: 0.8.0
xarray: None
IPython: 6.1.0
sphinx: None
patsy: 0.4.1
dateutil: 2.6.1
pytz: 2017.2
blosc: None
bottleneck: None
tables: None
numexpr: 2.6.2
matplotlib: 2.0.2
openpyxl: None
xlrd: 1.0.0
xlwt: None
xlsxwriter: None
lxml: None
bs4: 4.5.3
html5lib: 0.9999999
httplib2: None
apiclient: None
sqlalchemy: 1.1.13
pymysql: None
psycopg2: None
jinja2: 2.9.6
boto: None
pandas_datareader: None
None


# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [2]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [3]:
def get_list_of_university_towns():
    
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    
    df = pd.read_csv("university_towns.txt",delimiter="\n", header = None)
    df.columns = ['Data']
    
    states = (df[df['Data'].str
                 .find('[edit]') > 0])
    regions_states = pd.DataFrame(columns = [['State', 'RegionName']])

    st_id = (states.index
             .tolist())
    
    for i in range(len(st_id)):
        
        regions = pd.DataFrame(columns = [['State', 'RegionName']])
        
        if i < 49:
            
            start = st_id[i]
            end = st_id[i+1]
        
        if i == 49:
            
            start = st_id[i]
            end = df.shape[0]
        
        regions['RegionName']= df['Data'].iloc[start+1:end]
        regions['State']= df['Data'].iloc[start]
        
        regions_states = regions_states.append(regions)
    
    regions_states['State']= (regions_states['State'].str
                                    .split('(', expand=True)[0]
                                    .str
                                    .split('[', expand=True)[0]
                                    .str
                                    .rstrip())
    
    regions_states['RegionName'] = (regions_states['RegionName'].str
                                    .split('(', expand=True)[0]
                                    .str
                                    .split('[', expand=True)[0]
                                    .str
                                    .rstrip())
    
    regions_states.reset_index(inplace = True, drop = True)
    
    return regions_states

In [13]:
def create_gdp_df():
    
    df = pd.read_excel('gdplev.xls', header = 5, usecols=[3,5])
    df.columns = [['Quarters', 'GDP']]
    df = df[df['Quarters'] >= '2000q1']
    df = df.set_index('Quarters', drop = True).dropna()
    
    return df
create_gdp_df()

    Quarters      GDP
NaN   2000q1  12359.1
NaN   2000q2  12592.5
NaN   2000q3  12607.7
NaN   2000q4  12679.3
NaN   2001q1  12643.3


,GDP
Quarters,
2000q1,12359.1
2000q2,12592.5
2000q3,12607.7
2000q4,12679.3
2001q1,12643.3
2001q2,12710.3
2001q3,12670.1
2001q4,12705.3
2002q1,12822.3


In [16]:
def get_recession_start():
    
    
    df = create_gdp_df()

    for i in range(1, len(df)-3):

        if ((df['GDP'].iloc[i-1] > df['GDP'].iloc[i]) &
            (df['GDP'].iloc[i] > df['GDP'].iloc[i+1]) &
            (df['GDP'].iloc[i+1] < df['GDP'].iloc[i+2]) &
            (df['GDP'].iloc[i+2] < df['GDP'].iloc[i+3])):

            rec_id = i-1
            
    while df['GDP'].iloc[rec_id] < df['GDP'].iloc[rec_id-1]:
        
        rec_id -= 1

    return df.index[rec_id+1]
get_recession_start()

'2008q3'

In [18]:
def get_recession_end():
    
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    
    df = create_gdp_df()
    
    for i in range(1, len(df)-3):

        if ((df['GDP'].iloc[i-1] > df['GDP'].iloc[i]) &
            (df['GDP'].iloc[i] > df['GDP'].iloc[i+1]) &
            (df['GDP'].iloc[i+1] < df['GDP'].iloc[i+2]) &
            (df['GDP'].iloc[i+2] < df['GDP'].iloc[i+3])):

            rec_end = df.index[i+3]

    return rec_end
get_recession_end()

'2009q4'

In [19]:
def get_recession_bottom():
    
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    
    start = get_recession_start()
    end = get_recession_end()
    
    df = create_gdp_df()
    
    return df.loc[start:end,:].idxmin().tolist()[0]
get_recession_bottom()

'2009q2'

In [23]:
def convert_housing_data_to_quarters():
    
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    df_house = pd.read_csv('City_Zhvi_AllHomes.csv')
    
    df_house['State'] = df_house['State'].map(states)
    
    cols = [col for col in df_house.columns if col <= '1999-12']
    df_house.drop(cols, axis=1, inplace = True)
    
    df_house.set_index(['State', 'RegionName'], inplace = True)
        
    df_house.drop(['RegionID', 'Metro', 'CountyName', 'SizeRank'], axis=1, inplace = True)
           
    df_house.columns = pd.to_datetime(df_house.columns)
        
    df_house.columns = [(str(date.year) + 'q' + str(date.quarter)) for date in df_house.columns]
        
    df_house = (df_house.T.groupby(level = 0).agg(np.mean)).T.sort_index()
    
    return df_house

In [24]:
def run_ttest():
        
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    house_df = convert_housing_data_to_quarters()
    gdp_df = create_gdp_df()
    
    uni_towns = get_list_of_university_towns()
    uni_towns.set_index(['State', 'RegionName'], inplace = True)
    
    start = get_recession_start()
    end = get_recession_end()
    bottom = get_recession_bottom()
    before = house_df.columns.get_loc(get_recession_start())-1
    before = house_df.columns[before]
    
    cols = [col for col in house_df.columns if (col > end) | (col < before)]
    house_df.drop(cols, axis=1, inplace = True)
    
    house_df['RatioBB'] = (house_df[bottom] - house_df[before])/house_df[bottom]
    
    
    house_df.sort_index(inplace = True)
    
    uni_df = house_df[['RatioBB']].merge(uni_towns, right_index=True, left_index=True, how='inner').dropna()
    no_uni_df = house_df[['RatioBB']].drop(uni_df.index).dropna()  
    test = ttest_ind(uni_df['RatioBB'], no_uni_df['RatioBB']) 
    p = test[1]
    
    if p < 0.01:
        
        different = True
        better = "university town"
    else:
        different = False
        better = "non-university town"
    
    return (different, p, better)

In [25]:
run_ttest()

(True, 0.0027240637047612642, 'university town')